# Simulating Quorum Delegation on cadCAD

> *Danilo Lessa Bernardineli, October 2023*

## Part 1: Pre-requisites

In [ ]:
from dataclasses import dataclass
from functools import reduce
from typing import NamedTuple, TypedDict, Callable
from cadCAD_tools.types import *
from cadCAD_tools.execution import easy_run
from random import random, normalvariate, seed
from copy import deepcopy, copy

## Part 2: Module Logic

## Part 3: cadCAD Simulation

## Part 4: Results